In [1]:
import torch
import numpy as np
from utils.data import Data
from ner_model import build_model, evaluate
import time
import nltk
import os
import spacy
import spacy.displacy as  dp
import sys
%load_ext autoreload
%autoreload 2

In [9]:
tt = 'salut les aminches. Comment ça va ?'
nlp = spacy.load('en')

In [10]:
from utils.tokenizer import myTokenizer

In [11]:
tok = myTokenizer('fr')

In [4]:
path2text = '/home/matthieu/Documents/aphp/rapport/fic_CR.txt'
path2ann = '/home/matthieu/Documents/aphp/rapport/fic_CR.ann'

In [20]:
with open(path2ann, 'r') as f:
    labels = [l.split('\t')[1].split(' ') for l in f.read().splitlines()]
with open(path2text, 'r') as f:
    text = f.read()

In [ ]:
entities = [{'start':int(l[1]), 'end':int(l[2]), 'label':l[0]} for l in labels]
visu = [{'text':text, 'ents':entities, 'title':None}]
dp.serve(visu, style='ent', manual=True)


    Serving on port 5000...
    Using the 'ent' visualizer



127.0.0.1 - - [12/Aug/2018 13:13:52] "GET / HTTP/1.1" 200 9499
127.0.0.1 - - [12/Aug/2018 13:13:52] "GET /favicon.ico HTTP/1.1" 200 9499
127.0.0.1 - - [12/Aug/2018 13:13:52] "GET /favicon.ico HTTP/1.1" 200 9499


In [2]:
def align_data(data):
    ''' align a dictionnary of sequences 
        input:
            data, dict of sequences { 'key1': ['I', 'dream', 'of', 'the', 'Moon'] 
                        'key2': [O, O, O, O, 'B-LOC']}
        output:

            dict of strings {'key1': ['I dream of the Moon'] 
                             'key2': ['O O     O  O   B-LOC']}
    '''
    spacings = [max([len(seq[i]) for seq in data.values()]) for i in range(len(data[list(data.keys())[0]]))]

    data_aligned = dict()

    for key, seq in data.items():
        str_aligned = ""
        for token, spacing in zip(seq, spacings):
            str_aligned += token + " "*(spacing-len(token) + 1)
        data_aligned[key] = str_aligned
    return data_aligned
#print(align_data(align)['inputs'])
#print(align_data(align)['labels'])

In [3]:
path2xpt = '../pretrained/baseline.xpt'
path2model = '../pretrained/baseline.model'
decode_config_dict = {'load_model_dir':path2model # load model file
                     }
data = Data()
#data.read_config(decode_config_dict)
print("NER MODEL: decoding-style loading..")
## dset_dir must only contains dictionnary informations here (dset from the original model should be cleaned with the function clean_dset (to be coded))
data.load_export(path2xpt)
## supplementary configurations (optional, maybe not useful in deployment)
data.read_config(decode_config_dict)
## !!! we should be loading the weights here and not at each prediction!!!!
data.HP_gpu = torch.cuda.is_available()
#data.show_data_summary()
model = build_model(data)

NER MODEL: decoding-style loading..
Load Model weights from file ../pretrained/baseline.model
building Network..
use crf:  True
use_char:  True
char feature extractor:  CNN
word feature extractor:  LSTM
Build word sequence feature extractor: LSTM...
Build word representation...
Build char sequence feature extractor: CNN..
build CRF...


In [4]:
tokenizer = spacy.load('en', disable=['tagger','ner', 'parser'])
tokenizer.add_pipe(tokenizer.create_pipe('sentencizer')) 

## Test for one sentence

In [18]:
input_data = 'I love reading Camus.'
text = tokenizer(input_data)
input_client = []
input_model = []
for sent in text.sents:
    sentence = [token.string.strip() for token in sent]
    print(sentence)
    # we have to keep a sequence wo '' sentences separators for the client output
    input_client += sentence
    input_model += sentence + ['']

print(input_model)

start_time = time.time()
#print(feed_data)
data.generate_instance_from_list(input_model)
#print('***************')
#print(data.raw_texts)
#print(evaluate(data, model, 'raw', label_flag=False))
#print('*****************')
speed, acc, p, r, f, pred_results, pred_scores = evaluate(data, model, 'raw', label_flag=False) 

timed = time.time() - start_time
print('Processing time {:.2} s'.format(timed))
print('Decoding speed: {0:.2f} st/s'.format(speed))
# reconstruct a unique sequence for the client
live = True
if live: 
    output_client = []
    for l in pred_results:
        output_client += l
    output_aligned = align_data({'raw_input': input_client, 'labels':output_client})

    print(output_aligned['raw_input'])
    print(output_aligned['labels'])

['I', 'love', 'reading', 'Camus', '.']
['I', 'love', 'you', 'Juliette', '.']
['I', 'love', 'reading', 'Camus', '.', '', 'I', 'love', 'you', 'Juliette', '.', '']
Processing time 0.0049 s
Decoding speed: 437.96 st/s
I love reading Camus . I love you Juliette . 
O O    O       B-PER O O O    O   B-PER    O 


## test for a file

In [22]:
file_name = '../prod_data/wiki_en_france.txt'
out_folder = 'proprecessed/'
if not os.path.isdir(out_folder):
    os.mkdir(out_folder)
path2txt = out_folder + os.path.splitext(file_name)[0] + '.txt'
path2ann = out_folder + os.path.splitext(file_name)[0] + '.ann'
with open(file_name, 'r') as f:
    input_data = f.read()

text = tokenizer(input_data)
input_client = []
input_model = []
for sent in text.sents:
    sentence = [token.string.strip() for token in sent]
    #print(sentence)
    # we have to keep a sequence wo '' sentences separators for the client output
    input_client += sentence
    input_model += sentence + ['']

In [23]:
start_time = time.time()
#print(feed_data)
data.generate_instance_from_list(input_model)
#print('***************')
#print(data.raw_texts)
#print(evaluate(data, model, 'raw', label_flag=False))
#print('*****************')
speed, acc, p, r, f, pred_results, pred_scores = evaluate(data, model, 'raw', label_flag=False) 

timed = time.time() - start_time
print('Processing time {:.2} s'.format(timed))
print('Decoding speed: {0:.2f} st/s'.format(speed))
# reconstruct a unique sequence for the client
live = True
output_client = pred_results

Processing time 0.11 s
Decoding speed: 179.36 st/s


In [28]:
annotations = []
idx = 1
text = tokenizer(input_data)
for sent, label_seq in zip(text.sents, output_client):
    for token, label in zip(sent, label_seq):
        if (label != "O") :
            new_ann = 'T'+str(idx)+'\t'+label[2:]+' '+str(token.idx)+' '+str(token.idx+len(token.string.strip()))+'\t'+token.string.strip()+'\n'
            annotations.append(new_ann)
annotations

['T1\tLOC 0 6\tFrance\n',
 'T1\tMISC 8 14\tFrench\n',
 'T1\tLOC 41 47\tFrench\n',
 'T1\tLOC 48 56\tRepublic\n',
 'T1\tMISC 58 64\tFrench\n',
 'T1\tMISC 64 65\t:\n',
 'T1\tMISC 66 76\tRépublique\n',
 'T1\tLOC 170 176\tFrance\n',
 'T1\tLOC 180 187\tWestern\n',
 'T1\tLOC 188 194\tEurope\n',
 'T1\tORG 236 253\tterritories.[XIII\n',
 'T1\tLOC 280 286\tFrance\n',
 'T1\tLOC 304 317\tMediterranean\n',
 'T1\tLOC 318 321\tSea\n',
 'T1\tORG 329 336\tEnglish\n',
 'T1\tORG 337 344\tChannel\n',
 'T1\tLOC 353 358\tNorth\n',
 'T1\tLOC 359 362\tSea\n',
 'T1\tLOC 377 382\tRhine\n',
 'T1\tLOC 390 398\tAtlantic\n',
 'T1\tLOC 399 404\tOcean\n',
 'T1\tLOC 439 445\tFrench\n',
 'T1\tLOC 446 452\tGuiana\n',
 'T1\tLOC 456 461\tSouth\n',
 'T1\tLOC 462 469\tAmerica\n',
 'T1\tLOC 497 505\tAtlantic\n',
 'T1\tLOC 507 514\tPacific\n',
 'T1\tMISC 519 525\tIndian\n',
 'T1\tLOC 735 741\tFrance\n',
 'T1\tLOC 802 807\tParis\n',
 'T1\tORG 911 920\tMarseille\n',
 'T1\tORG 922 926\tLyon\n',
 'T1\tORG 928 933\tLille\n',
 'T1\

In [13]:
#input_data = 'I am working at the APHP. They have recently refused Google and Facebook cooperation. Camus wrote such beautiful plays'
## Pre-processing from client 
sentences = nltk.sent_tokenize(input_data)
input_client = []
input_model = []
for sent in sentences:
    tokens = nltk.word_tokenize(sent)
    # we have to keep a sequence wo '' sentences separators for the client output
    input_client += tokens
    input_model += tokens + ['']
#print(input_client)
#print(input_model)

In [21]:
start_time = time.time()
#print(feed_data)
### self.fix_alphabet() placed inside generate_instance* should prevent the vocabularies to grow indefinitely with fed inputs
data.generate_instance_from_list(input_model)
#print('***************')
#print(data.raw_texts)
#print(evaluate(data, model, 'raw', label_flag=False))
#print('*****************')
speed, acc, p, r, f, pred_results, pred_scores = evaluate(data, model, 'raw', label_flag=False) 

timed = time.time() - start_time
print('Processing time {:.2} s'.format(timed))
print('Decoding speed: {0:.1f} st/s'.format(speed))
print(pred_results)
# reconstruct a unique sequence for the client
#output_client = []
#for l in pred_results:
#    output_client += l

#output_aligned = align_data({'raw_input': input_client, 'labels':output_client})
#print(output_aligned['raw_input'])
#print(output_aligned['labels'])
out = [' '.join(sent) +'\n' for sent in pred_results]
print(out)
with open(path2write, 'w') as f:
    f.writelines(out)
    

Processing time 0.042 s
Decoding speed: 747.7 st/s
[['B-LOC', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'B-LOC', 'I-LOC', 'O'], ['O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-MISC', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

## Decoding tests

In [5]:
path2gold = '../conll2003/test.conll2003'
path2decode = '../pretrained/myModel/myModel.out'
path2dset = '../pretrained/myModel/myModel.dset'
path2model = '../pretrained/myModel/myModel.0.model'

conf_dict = {'raw_dir':path2gold,
                   'decode_dir':path2decode,
                    'dset_dir':path2dset,
                    'load_model_dir':path2model,
                    'number_normalized': True
                   }

In [6]:
from utils.data import Data
from ner_model import load_model_decode

data = Data()
data.read_config(conf_dict)
data.load(data.dset_dir)
data.read_config(conf_dict)
data.HP_gpu = torch.cuda.is_available()
data.generate_instance('raw')
print(len(data.raw_texts))
print(len(data.test_texts))

3683
3683


In [7]:
decode_results, _ = load_model_decode(data, 'raw', label_flag=True)

Load Model from file ../pretrained/myModel/myModel
building Network..
use crf:  True
use_char:  True
char feature extractor:  CNN
word feature extractor:  LSTM
Build word sequence feature extractor: LSTM...
Build word representation...
Build char sequence feature extractor: CNN..
build CRF...
gold_num = 5596; predict_num = 5696; right_num = 4683
raw: time10.67s, speed: 349.10st/s; acc: 0.9656, p: 0.8222, r: 0.8368, f: 0.8294


In [8]:
print(len(decode_results))

3683


10

In [11]:
data.write_decoded_results(decode_results, 'raw')

Predict raw result has been written into file ../pretrained/myModel/myModel.out
